In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType

import pandas

## 全データを使ったモデリング（流れの確認）
- 特徴量には数値の絡むのみ仕様する
- 標準化は行わない
- one-hot encodingは行わない
- 重複行などのデータチェックは行わない

In [16]:
spark = SparkSession.builder.appName('Data_wrangling').getOrCreate()

In [17]:
data = spark.read.format("csv")\
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ";") \
    .load("./data/bank-full.csv")

In [18]:
data.summary().show()

+-------+------------------+-------+--------+---------+-------+------------------+-------+-----+--------+-----------------+-----+-----------------+------------------+------------------+------------------+--------+-----+
|summary|               age|    job| marital|education|default|           balance|housing| loan| contact|              day|month|         duration|          campaign|             pdays|          previous|poutcome|    y|
+-------+------------------+-------+--------+---------+-------+------------------+-------+-----+--------+-----------------+-----+-----------------+------------------+------------------+------------------+--------+-----+
|  count|             45211|  45211|   45211|    45211|  45211|             45211|  45211|45211|   45211|            45211|45211|            45211|             45211|             45211|             45211|   45211|45211|
|   mean| 40.93621021432837|   null|    null|     null|   null|1362.2720576850766|   null| null|    null|15.806418791886

In [19]:
# データ作成
linear_df = data.select(["age", "balance", "campaign"])
target = "balance"
features = ["age", "campaign"]
train_df = data.select(features)

In [20]:
# データ作成ステージ
from pyspark.ml.feature import VectorAssembler
assemble = VectorAssembler(inputCols=features, outputCol="features")

In [21]:
# 線形重回帰モデリングステージ
from pyspark.ml.regression import LinearRegression
clf = LinearRegression(featuresCol="features", labelCol="balance")

In [22]:
# パイプラインの設定：ステージの登録
from pyspark.ml.pipeline import Pipeline
pipeline = Pipeline(stages=[assemble, clf])
model = pipeline.fit(linear_df)

23/02/26 23:27:31 WARN Instrumentation: [fa46899b] regParam is zero, which might cause numerical instability and overfitting.


In [24]:
# パイプラインの実行
df = model.transform(linear_df)
df.show()

+---+-------+--------+----------+------------------+
|age|balance|campaign|  features|        prediction|
+---+-------+--------+----------+------------------+
| 58|   2143|       1|[58.0,1.0]|1867.1309208276969|
| 44|     29|       1|[44.0,1.0]| 1474.315799038966|
| 33|      2|       1|[33.0,1.0]| 1165.675346204963|
| 47|   1506|       1|[47.0,1.0]| 1558.490467993694|
| 33|      1|       1|[33.0,1.0]| 1165.675346204963|
| 35|    231|       1|[35.0,1.0]|1221.7917921747817|
| 28|    447|       1|[28.0,1.0]|1025.3842312804163|
| 42|      2|       1|[42.0,1.0]|1418.1993530691473|
| 58|    121|       1|[58.0,1.0]|1867.1309208276969|
| 43|    593|       1|[43.0,1.0]|1446.2575760540565|
| 41|    270|       1|[41.0,1.0]|1390.1411300842378|
| 29|    390|       1|[29.0,1.0]|1053.4424542653255|
| 53|      6|       1|[53.0,1.0]|1726.8398059031501|
| 58|     71|       1|[58.0,1.0]|1867.1309208276969|
| 57|    162|       1|[57.0,1.0]|1839.0726978427874|
| 51|    229|       1|[51.0,1.0]|1670.72335993

In [27]:
# 係数の確認
print(model.stages[1].coefficients)
# 切片の確認
print(model.stages[1].intercept)

[28.058222984909357,-14.785487706439227]
254.53947540939342
